In [ ]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
print('packages imported')


Getting the Wikipedia URL

In [ ]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response=requests.get(url)

html_soup=BeautifulSoup(response.text,'html.parser')
print(html_soup)

Getting table Values

In [ ]:
table=html_soup.find('table',class_='wikitable sortable')
print(table)

In [ ]:
row=table.find('tr')
print(row)

In [ ]:
data=[]

rows=table.find_all('tr')
for row in rows:
    row_data=row.find('td')
    row_full=row.find_all('td')
    data_sp=[]
    for row_data in row_full:
        val=row_data.get_text()
        val=val[:-1]
        data_sp.append(val)
    
    data.append(data_sp)
    
data=data[1:]
print(data)
        

Creating Dataframe

In [ ]:
l=len(data)
data_new=[]
for i in range(0,l):
    if(data[i][1] != 'Not assigned'):
        data_new.append(data[i])

dataframe=pd.DataFrame(data_new,columns=['PostalCode','Borough','Neighbourhood'])
dataframe

In [ ]:
l=len(dataframe)
error_correct=dataframe.iloc[:,2].values
for i in range(0,l):
    corr=error_correct[i].replace('/',',')
    error_correct[i]=corr
    
dataframe

In [ ]:
dataframe.shape

# Finding the Latitudes and Longitudes of Each Location

In [ ]:
df_coordinates=pd.read_csv('Geospatial_Coordinates.csv')
df_coordinates

In [ ]:
lat=[]
long=[]
l=len(dataframe)
pc_codes=df_coordinates.iloc[:,0].values
pc_lat=df_coordinates.iloc[:,1].values
pc_long=df_coordinates.iloc[:,2].values

postal_codes=dataframe.iloc[:,0].values

for i in range(0,l):
    a=postal_codes[i]
    for j in range(0,l):
        if(a==pc_codes[j]):
            lat.append(pc_lat[j])
            long.append(pc_long[j])
            break
            
dataframe['Latitude']=lat
dataframe['Longitude']=long

dataframe
    
    

# Making of Clusters

In [ ]:
toronto_data=[]
dfd=dataframe.iloc[:,:].values
df_borough=dataframe.iloc[:,1].values
str='Toronto'
for i in range(0,l):
    index=df_borough[i].find(str)
    if(index != -1):
        toronto_data.append(dfd[i])
        
toronto_df=pd.DataFrame(toronto_data,columns=['PostalCode','Borough','Neighborhood','Latitude','Longitude'])


In [ ]:
toronto_df

# Applying same method that was in Newyork's Lab

In [ ]:
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libs Imported')

In [ ]:
address = 'Toronto City'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

In [ ]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'],toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [ ]:
CLIENT_ID = 'QHR03ZCNADFBJXGBSQ1X3TU1RJO1SMHW5UOPI1NR0WCTPJPP' # your Foursquare ID
CLIENT_SECRET = 'VK3BPVULUIWLLSY51DYUTT3ACHVVB2XIW5SSNMZ4EDXOT0L5' # your Foursquare Secret
VERSION = '20200425' # Foursquare API version
LIMIT=100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:

def getNearbyVenues(names, latitudes, longitudes, radius=200):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
toronto_venues = getNearbyVenues(names=toronto_df['Neighborhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )


In [ ]:
toronto_venues

In [ ]:
toronto_venues.groupby('Neighborhood').count()

In [ ]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

dummy=toronto_venues.iloc[:,0].values

toronto_onehot.insert(0,'neighborhood',dummy)

toronto_onehot.head()

In [ ]:
toronto_grouped = toronto_onehot.groupby('neighborhood').mean().reset_index()
toronto_grouped

In [ ]:
toronto_grouped.shape

In [ ]:
num_top_venues = 5

for hood in toronto_grouped['neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

In [ ]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'cluster labels', kmeans.labels_)

neighborhoods_venues_sorted

In [ ]:
toronto_merged=neighborhoods_venues_sorted
toronto_merged.insert(0,'Longitude','')
toronto_merged.insert(0,'Latitude','')
toronto_merged

In [ ]:
l1=len(toronto_merged)
l2=len(toronto_df)

arr_lat=toronto_merged.iloc[:,0].values
arr_long=toronto_merged.iloc[:,1].values

for i in range(0,l1):
    a=toronto_merged['Neighborhood'][i]
    for j in range(0,l2):
        if(toronto_df['Neighborhood'][j]==a):
            arr_lat[i]=toronto_df['Latitude'][j]
            arr_long[i]=toronto_df['Longitude'][j]
            break
            
toronto_merged
    

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['cluster labels']):
    label = folium.Popup(poi + ' Cluster '  , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters